In [1]:
import os
import re
from dotenv import load_dotenv
load_dotenv()
from typing import TypeVar, NamedTuple
T = TypeVar('T')
from kv.api import KV
from kv.sql import SQLKV
from kv.azure.blob import BlobKV, BlobContainerKV
from sqlalchemy import create_engine

In [2]:
conn_str = os.environ['BLOB_CONN_STR']